## 混合精确训练
该模块允许神经网络的前向和后向传递以fp16（也称为半精度）完成。 如果您拥有带有张量内核的NVIDIA GPU，这一点尤为重要，因为它可以将您的培训速度提高200％或更多。

## Overview
要以混合精度训练模型，您只需调用Learner.to_fp16，它会转换模型并修改现有的Learner以添加MixedPrecision。

<b>to_fp16</b>

`to_fp16(learn:Learner, loss_scale:float=None, max_noskip:int=1000, dynamic:bool=True, clip:float=None, flat_master:bool=False, max_scale:float=16777216) → Learner`

学习FP16精度模式。

例如：

In [1]:
from fastai.vision import *
from fastai.basics import *
from fastai.callbacks import *

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy]).to_fp16()
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.146248,0.125436,0.956820,01:00


有关混合精确培训的详细信息，请参见NVIDIA的文档。 我们将在此总结基础知识。

您可能想要调整的唯一参数是`loss_scale`。 这用于缩放损耗，因此它不会下溢fp16，导致精度损失（这在转换回fp32后的最终梯度计算中相反）。 通常，默认`512`运行良好。 您也可以使用`flat_master = True`启用或禁用主参数张量的展平，但在我们的测试中，差异可以忽略不计。

在内部，回调确保所有模型参数（除了batchnorm图层，需要fp32）都转换为fp16，并且还保存了fp32副本。 fp32副本（主参数）用于使用优化器进行实际更新; fp16参数用于计算梯度。 这有助于避免学习率较低的下溢。

所有这些都是通过以下Callback实现的。

### class MixedPrecision

`MixedPrecision(learn:Learner, loss_scale:float=None, max_noskip:int=1000, dynamic:bool=True, clip:float=None, flat_master:bool=False, max_scale:float=16777216) :: LearnerCallback`

用于为学习者创建回调的基类。

### 回调方法
您不必自己调用以下函数 -  fastai的Callback系统会自动调用它们来启用类的功能。

<b>on_backward_begin</b>

`on_backward_begin(last_loss:Rank0Tensor, **kwargs:Any) → Rank0Tensor`

通过self.loss_scale缩放比例以防止下溢。

<b>on_backward_end</b>

`on_backward_end(**kwargs:Any)`

将渐变转换回FP32并按比例划分它们。

<b>on_loss_begin</b>

`on_loss_begin(last_output:Tensor, **kwargs:Any) → Tensor`

将半精度输出转换为FP32以避免减少溢出。

<b>on_step_end</b>

`on_step_end(**kwargs:Any)`

更新从主模型到模型和零毕业的参数。

<b>on_train_begin</b>

`on_train_begin(**kwargs:Any)`

准备主模型。